In [173]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [174]:
url = "https://berlinerhm.r.mikatiming.net/2023/?page={}&event=HML&pid=search"
no_pages = 1052
dfs = []


In [175]:
for i in range(1, no_pages + 1):
    page = url.format(i)
    raw = requests.get(url.format(i)).content
    parsed_html = BeautifulSoup(raw)

    # remove labels
    for label in parsed_html.find_all(attrs={'class': 'list-label'}):
        label.extract()

    entries = parsed_html.body.find_all('li', attrs={'class':'list-group-item'})

    data = []
    for e in entries:
        row = [i.text for i in e.find_all(attrs={'class': 'list-field'})]
        if len(row) > 0:
            data.append(row)
    
    dfs.append(pd.DataFrame(data[1:], columns=data[0]))
    time.sleep(0.1)


In [211]:
df = (
    pd.concat(dfs)
    .reset_index(drop=True)
    .replace('–', '-')
)
df = df.loc[:, df.columns != '']
df


,Platz,Pl.AK,Name,Startnr.,AK,Verein,Netto,Brutto
0,7568,769,(GER),21455,M45,-,01:52:33,02:26:02
1,13182,2301,"à Brassard, Jérôme (GER)",12002,MH,-,02:14:47,03:25:17
2,907,194,"Aabye, Simone (DEN)",50002,W30,Hellas,01:43:02,01:47:31
3,13970,1481,"Aaen, Lasse (DEN)",12003,M45,-,02:20:53,03:10:27
4,1498,76,"Aakerhjelm, Pia (DEN)",50003,W50,Puls 8410,01:48:39,01:52:02
...,...,...,...,...,...,...,...,...
26277,9843,2674,"Østlie, Martha (NOR)",29561,WH,-,02:45:09,03:37:31
26278,2367,559,"Østrup-Møller, Joakim (DEN)",29564,MH,SOS Running,01:33:38,01:36:23
26279,157,71,"Øvre-Helland, Sondre (NOR)",29651,MH,TIF Viking,01:12:02,01:12:26
26280,1243,485,"Øvstetun, Silje (NOR)",29652,WH,-,01:46:23,02:25:54


In [212]:
df.to_csv("./data.csv", sep=';')
